<a href="https://colab.research.google.com/github/slaninam/MCDA/blob/master/exploratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import urllib.request
import zipfile

from pathlib import Path

datafile = Path('data.zip')

if not datafile.exists():
  print('Retrieving data file')
  url = 'https://m-cloud.netztest.at/RMBTStatisticServer/export/netztest-opendata-2019-09.zip'
  urllib.request.urlretrieve(url, 'data.zip')

print(datafile.exists())




#with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
#    zip_ref.extractall(directory_to_extract_to)

True


In [11]:
ls sample_data


anscombe.json*                mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md*
